In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/focus/Study_For_DeepLearning

/content/drive/MyDrive/focus/Study_For_DeepLearning


In [3]:
!git config --global user.email 'minsubrother@naver.com'
!git config --global user.name 'HongkyuRyu'

In [4]:
!git add catndog.ipynb

fatal: pathspec 'catndog.ipynb' did not match any files
